<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>IA Generativa e LLMs Para Processamento de Linguagem Natural</font>
## <font color='blue'>Projeto 5</font>
## <font color='blue'>LangChain e LLM Open-Source Para Sistema de Perguntas e Respostas</font>

## Instalando e Carregando Pacotes

In [1]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 75.0 MB/s eta 0:00:00


In [2]:
!pip install -q accelerate peft bitsandbytes transformers trl datasets langchain langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.6/504.6 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
# Imports
import torch
import accelerate
import peft
import bitsandbytes
import transformers
import trl
import datasets
import langchain
import langchain_community

In [4]:
# Imports
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import pipeline, TrainingArguments
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



## Carregando o Dataset Para o Instruction Fine-Tuning

https://huggingface.co/datasets/nlpie/Llama2-MedTuned-Instructions

In [6]:
# Carrega o dataset
dataset = load_dataset("nlpie/Llama2-MedTuned-Instructions")

README.md: 0.00B [00:00, ?B/s]

(…)-00000-of-00001-a8790d88efc2bc45.parquet:   0%|          | 0.00/91.1M [00:00<?, ?B/s]

(…)-00000-of-00001-b543c64b1786c03e.parquet:   0%|          | 0.00/6.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/200252 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/70066 [00:00<?, ? examples/s]

In [7]:
# Selecionamos as linhas para treino do modelo
dsa_dados_treino = dataset["train"].select(indices = range(1000))

In [8]:
dsa_dados_treino

Dataset({
    features: ['instruction', 'input', 'output', 'source'],
    num_rows: 1000
})

In [9]:
# Selecionamos as linhas para teste do modelo
dsa_dados_teste = dataset["train"].select(indices = range(1000, 1200))

In [10]:
dsa_dados_teste

Dataset({
    features: ['instruction', 'input', 'output', 'source'],
    num_rows: 200
})

## Compreendendo o Formato dos Dados de Texto

In [11]:
# Vamos visualizar 3 pontos de dados
for i in range(3):
    data = dataset['train'][i]
    print(f"Ponto de Dado {i + 1}:")
    print("Instruction:", data['instruction'])
    print("Input:", data['input'])
    print("Output:", data['output'])
    print("\n-----------------------------\n")

Ponto de Dado 1:
Instruction: In your role as a medical professional, address the user's medical questions and concerns.
Input: My relative suffering from secondary lever cancer ( 4th stage as per Allopathic doctor) and primary is in rectum. He is continuously with 103 to 104 degree F fever. Allpathic doctor suggested chemo only after fever subsidises. Is treatment possible at Lavanya & what is the time scale of recover.
Output: Hi, dairy have gone through your question. I can understand your concern. He has rectal cancer with liver metastasis. It is stage 4 cancer. Surgery is not possible at this stage. Only treatment options are chemotherapy and radiotherapy according to type of cancer. Inspite of all treatment prognosis is poor. Life expectancy is not good. Consult your doctor and plan accordingly. Hope I have answered your question, if you have any doubts then contact me at bit.ly/ Chat Doctor. Thanks for using Chat Doctor. Wish you a very good health.

----------------------------

## Automatizando a Criação dos Prompts Para Treinamento do Modelo

In [12]:
# Define a função que recebe um dicionário chamado sample
def dsa_cria_prompt(sample):

    # Define uma string pre_prompt que serve como um modelo para a primeira parte do prompt
    pre_prompt = """[INST]<<SYS>> {instruction}\n"""

    # Concatena o pre_prompt com strings adicionais para formar o prompt completo
    prompt = pre_prompt + "{input}" +"[/INST]"+"\n{output}"

    # Atribui o valor da chave 'instruction' do dicionário sample à variável example_instruction
    example_instruction = sample['instruction']

    # Atribui o valor da chave 'input' do dicionário sample à variável example_input
    example_input = sample['input']

    # Atribui o valor da chave 'output' do dicionário sample à variável example_output
    example_output = sample['output']

    # Cria uma instância de PromptTemplate com o prompt definido anteriormente e as variáveis de entrada
    prompt_template = PromptTemplate(template = prompt,
                                     input_variables = ["instruction", "input", "output"])

    # Utiliza o método format da instância prompt_template para substituir as variáveis
    # no template com os valores específicos
    prompt_unico = prompt_template.format(instruction = example_instruction,
                                          input = example_input,
                                          output = example_output)

    # Retorna o prompt formatado
    return prompt_unico

In [13]:
# Testando a função
prompt = dsa_cria_prompt(dsa_dados_treino[0])
print(prompt)

[INST]<<SYS>> In your role as a medical professional, address the user's medical questions and concerns.
My relative suffering from secondary lever cancer ( 4th stage as per Allopathic doctor) and primary is in rectum. He is continuously with 103 to 104 degree F fever. Allpathic doctor suggested chemo only after fever subsidises. Is treatment possible at Lavanya & what is the time scale of recover.[/INST]
Hi, dairy have gone through your question. I can understand your concern. He has rectal cancer with liver metastasis. It is stage 4 cancer. Surgery is not possible at this stage. Only treatment options are chemotherapy and radiotherapy according to type of cancer. Inspite of all treatment prognosis is poor. Life expectancy is not good. Consult your doctor and plan accordingly. Hope I have answered your question, if you have any doubts then contact me at bit.ly/ Chat Doctor. Thanks for using Chat Doctor. Wish you a very good health.


In [14]:
# Testando a função
prompt = dsa_cria_prompt(dsa_dados_teste[0])
print(prompt)

[INST]<<SYS>> In the clinical text, your objective is to identify relationships between medical problems, treatments, and tests. Medical problems are tagged as @problem$, medical tests as @test$, and treatments as @treatment$. Classify the relationship between two entities as one of the following:
Treatment improves medical problem (TrIP)
Treatment worsens medical problem (TrWP)
Treatment causes medical problem (TrCP)
Treatment is administered for medical problem (TrAP)
Treatment is not administered because of medical problem (TrNAP)
Test reveals medical problem (TeRP)
Test conducted to investigate medical problem (TeCP)
Medical problem indicates medical problem (PIP)
No Relations
Include @treatment$ 50 mgs bid , Aricept 10 mgs qhs , @treatment$ 15 mgs bid , Trazodone 100 mgs qhs .[/INST]
No Relations


## Processo de Quantização

In [15]:
# Ativa o carregamento do modelo base com precisão de 4 bits
use_4bit = True

In [16]:
# Define o dtype para o modelo base
bnb_4bit_compute_dtype = "float16"

In [17]:
# Tipo de quantização
bnb_4bit_quant_type = "nf4"

In [18]:
# Desativa a quantização dupla
use_nested_quant = False

In [19]:
# Define o dtype para computação no PyTorch
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

In [20]:
# Define as configurações
bnb_config = BitsAndBytesConfig(load_in_4bit = use_4bit,
                                bnb_4bit_quant_type = bnb_4bit_quant_type,
                                bnb_4bit_compute_dtype = compute_dtype,
                                bnb_4bit_use_double_quant = use_nested_quant)

In [21]:
# Verifica se a GPU suporta bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("A GPU suporta bfloat16. Acelere o treinamento usando bf16=True")
        print("=" * 80)

A GPU suporta bfloat16. Acelere o treinamento usando bf16=True


## Carregando LLM e Tokenizador

https://huggingface.co/NousResearch/Llama-2-7b-chat-hf

In [22]:
# Nome do LLM
nome_llm = "NousResearch/Llama-2-7b-chat-hf"

In [23]:
# Carrega o tokenizador
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [24]:
# Carrega o modelo base com a quantização
modelo = AutoModelForCausalLM.from_pretrained(nome_llm,
                                              quantization_config = bnb_config,
                                              device_map = "auto",
                                              use_cache = False)

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [25]:
# Usa o token EOS do tokenizador para o pad ao fim de cada sequência
tokenizer.pad_token = tokenizer.eos_token

In [26]:
# Ativa o padding ao final de cada sentença
tokenizer.padding_side = "right"

## Configurando Adaptadores LoRa

In [27]:
# Parâmetros LoRa
dsa_peft_config = LoraConfig(r = 8,
                             lora_alpha = 16,
                             lora_dropout = 0.05,
                             bias = "none",
                             task_type = "CAUSAL_LM")

A quantização representa dados com menos bits, tornando-se uma técnica útil para reduzir o uso de memória e acelerar a inferência, especialmente quando se trata de LLMs.

Depois que um modelo é quantizado, ele normalmente não é treinado DIRETAMENTE para tarefas posteriores porque o treinamento pode ser instável devido à menor precisão dos pesos e ativações. Mas como os métodos PEFT apenas adicionam parâmetros extras treináveis, isso permite treinar um modelo quantizado com um adaptador PEFT na parte superior! Combinar quantização com PEFT pode ser uma boa estratégia para treinar até mesmo os maiores modelos em uma única GPU. Por exemplo, QLoRA é um método que quantiza um modelo em 4 bits e depois o treina com LoRA. Este método permite ajustar um modelo de parâmetros de 65B em uma única GPU de 48GB, por exemplo.

O objetivo do PEFT (Parameter-Efficient Fine-Tuning) é manter a maioria dos parâmetros do modelo pré-treinado fixos e ajustar apenas um pequeno subconjunto de parâmetros para adaptar o modelo a uma tarefa específica.

In [28]:
# Prepara o modelo para treinamento
modelo_dsa = prepare_model_for_kbit_training(modelo)

In [29]:
# Junta o modelo quantizado com os adaptadores LoRa
modelo_dsa = get_peft_model(modelo_dsa, dsa_peft_config)

## Parâmetros do Ajuste Fino

In [30]:
output_model = "modelo_ajustado"

In [31]:
# Configuração de TrainingArguments
dsa_training_arguments = TrainingArguments(output_dir = output_model,
                                           per_device_train_batch_size = 1,
                                           gradient_accumulation_steps = 4,
                                           optim = "paged_adamw_32bit",
                                           learning_rate = 2e-4,
                                           lr_scheduler_type = "cosine",
                                           save_strategy = "epoch",
                                           logging_steps = 10,
                                           num_train_epochs = 3,
                                           max_steps = 150,
                                           fp16 = True,
                                           report_to = "none")

In [32]:
# Configuração do SFTConfig somente com argumentos próprios (requerido nas versões mais recentes do pacote trl)
sft_config = SFTConfig(packing = True,
                       dataset_text_field = "instruction")  # O campo base usado para criar prompts

In [33]:
# Criação do SFTTrainer
dsa_trainer = SFTTrainer(model = modelo_dsa,
                         args = dsa_training_arguments,
                         train_dataset = dsa_dados_treino,
                         eval_dataset = dsa_dados_teste,
                         peft_config = dsa_peft_config,
                         formatting_func = dsa_cria_prompt)

Applying formatting function to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Applying formatting function to eval dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## Ajuste Fino do LLM

Nota: Se solicitado, faça o cadastro em https://wandb.ai/authorize e use sua API na célula abaixo.

In [34]:
%%time
dsa_trainer.train()

Step,Training Loss
10,2.656300
20,2.383300
30,1.997800
40,1.602600
50,1.614700
60,1.642600
70,1.463200
80,1.326900
90,1.519600
100,1.380400


CPU times: user 3min 56s, sys: 272 ms, total: 3min 56s
Wall time: 3min 56s


TrainOutput(global_step=150, training_loss=1.5996423212687174, metrics={'train_runtime': 236.2135, 'train_samples_per_second': 2.54, 'train_steps_per_second': 0.635, 'total_flos': 6783517244375040.0, 'train_loss': 1.5996423212687174})

In [35]:
# Salva o modelo
dsa_trainer.save_model("modelo_final")

In [36]:
# Merge
merged_model = modelo_dsa.merge_and_unload()

## Construindo o Pipeline de Geração de Texto com LangChain

In [37]:
# Cria o pre-prompt com a instrução
pre_prompt = """[INST] <<SYS>>\nAnalyze the question and answer with the best option.\n"""

In [38]:
# Cria o prompt adicionando o input
prompt = pre_prompt + "Here is my question {context}" + "[\INST]"

In [39]:
# Cria o prompt template com LangChain
prompt = PromptTemplate(template = prompt, input_variables = ["context"])

Os pipelines são uma maneira excelente e fácil de usar modelos para inferência. Esses pipelines são objetos que abstraem a maior parte do código complexo da biblioteca, oferecendo uma API simples dedicada a diversas tarefas, incluindo reconhecimento de entidade nomeada, modelagem de linguagem mascarada, análise de sentimento, extração de recursos e resposta a perguntas.

In [40]:
# Cria o objeto pipeline
dsa_pipe = pipeline("text-generation",
                    model = merged_model,
                    tokenizer = tokenizer,
                    max_new_tokens = 512,
                    use_cache = False,
                    do_sample = True,
                    pad_token_id = tokenizer.eos_token_id,
                    top_p = 0.7,
                    temperature = 0.5)

Device set to use cuda:0


In [41]:
# Cria o Hugging Face Pipeline
llm_pipeline = HuggingFacePipeline(pipeline = dsa_pipe)

## Criando a LLM Chain

In [42]:
# Cria a memória
memory = ConversationBufferMemory()

In [43]:
# Cria o LLM Chain
dsa_chat_llm_chain = LLMChain(llm = llm_pipeline,
                              prompt = prompt,
                              verbose = False,
                              memory = memory)

## Deploy do Modelo e Uso do Sistema de Perguntas e Respostas

In [44]:
contexto = '''###Question: All of the following provisions are included in the Primary health care according to the Alma Ata declaration except:
###Options:
A. Adequate supply of safe drinking water
B. Nutrition
C. Provision of free medicines
D. Basic sanitation'''

In [45]:
%%time
dsa_chat_llm_chain.predict(context = contexto)

CPU times: user 27.3 s, sys: 169 ms, total: 27.5 s
Wall time: 27.5 s


'[INST] <<SYS>>\nAnalyze the question and answer with the best option.\nHere is my question ###Question: All of the following provisions are included in the Primary health care according to the Alma Ata declaration except:\n###Options:\nA. Adequate supply of safe drinking water\nB. Nutrition\nC. Provision of free medicines\nD. Basic sanitation[\\INST]  The best answer is (D) Basic sanitation.\n\nThe Alma Ata Declaration, signed in 1978, is a landmark document that outlines the principles of primary health care. According to the declaration, primary health care should include the following provisions:\n\n* Adequate supply of safe drinking water (Option A)\n* Nutrition (Option B)\n* Provision of free medicines (Option C)\n\nHowever, the declaration does not mention basic sanitation as one of the provisions of primary health care. Therefore, the best answer is (D) Basic sanitation.'

In [46]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [47]:
#%watermark -v -m

In [48]:
#%watermark --iversions

# Fim